In [11]:
import pyvista as pv
import tetgen
from pymeshfix import MeshFix

# Define the path to the STL file
stl_file_path = r'C:\Users\Bence\Documents\Bike Stem S1 Copy.stl'

# Read the STL file using PyVista
mesh = pv.read(stl_file_path)

# Ensure the mesh is manifold by using PyVista's cleaning functions
clean_mesh = mesh.clean()

# Further repair the mesh using PyMeshFix
# Convert to a format suitable for PyMeshFix (vertices and faces)
points = clean_mesh.points
faces = clean_mesh.faces.reshape(-1, 4)[:, 1:4]  # remove the first element of each face which is the number of points in the face

# Initialize MeshFix
meshfix = MeshFix(points, faces)

# Perform the repair
meshfix.repair(verbose=True)

# Extract the repaired mesh
fixed_mesh = meshfix.mesh

# Convert the repaired mesh back to a PyVista PolyData object
repaired_mesh = pv.PolyData(fixed_mesh.points, fixed_mesh.faces)

# Ensure the repaired mesh is cleaned and triangulated
repaired_mesh = repaired_mesh.clean().triangulate()

# Convert the cleaned and repaired mesh to a format suitable for TetGen
tet = tetgen.TetGen(repaired_mesh)

try:
    # Attempt to tetrahedralize the repaired mesh
    tetra_mesh = tet.tetrahedralize(
        order = 1,
        mindihedral = 10,
        minratio = 1.5,
        quality = True,
        steinerleft = -1,
    )
    print("Tetrahedralization successful.")
    
except RuntimeError as e:
    print(f"Failed to tetrahedralize: {e}")
    print("May need to further repair surface by making it manifold.")


Fixing degeneracies and intersections
Tetrahedralization successful.


In [12]:
from sigmaepsilon.mesh import PolyData, PointData
from sigmaepsilon.mesh.space import StandardFrame
from sigmaepsilon.mesh.cells import TET4

coords, topo = tetra_mesh
frame = StandardFrame(dim=3)
mesh = PolyData(pd=PointData(coords=coords, frame=frame), cd=TET4(topo=topo, frames=frame))
mesh.plot(notebook=False)